<a href="https://colab.research.google.com/github/1967-pepper/Fin-Analysis/blob/main/Stockappend_Fundamentals_BeautySoup_AppendedSt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import json
import csv
from io import StringIO
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Bitacora:
*   Consider to use an average of several days as final prices
*   Look for another method to append DF's to avoid drop NA's
*   IDK why x.drop works well at stmts joining but not with the rest
*   





In [ ]:
url_financials = 'https://finance.yahoo.com/quote/{}/financials?p={}'

In [ ]:
def main_scraper(stock):

      print('Attempting to scrape data for ' + stock)
      response = requests.get(url_financials.format(stock, stock))

      soup = BeautifulSoup(response.text, 'html.parser')
      #
      pattern = re.compile(r'\s--\sData\s--\s')
      script_data = soup.find('script', text=pattern).contents[0]
      # beginning
      print(script_data[:500])
      # The end
      print(script_data[-500:])
      start = script_data.find("context")-2
      json_data = json.loads(script_data[start:-12])

      print(json_data['context'].keys())
      print(json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys())

      annual_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']
      quarterly_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistoryQuarterly']['incomeStatementHistory']

      annual_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistory']['cashflowStatements']
      quarterly_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistoryQuarterly']['cashflowStatements']

      annual_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']
      quarterly_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistoryQuarterly']['balanceSheetStatements']


      annual_stmts = []
      all_st = [annual_is, annual_bs, annual_cf]
      for each_st in all_st:
        for s in each_st:
            statement = {}
            for key, val in s.items():
                try:
                    statement[key] = val['raw']
                except TypeError:
                      continue
                except KeyError:
                      continue
            annual_stmts.append(statement)


            app_raw_st = pd.DataFrame(annual_stmts)
            app_raw_st = app_raw_st.apply(lambda x: pd.Series(x.dropna().values))
            app_raw_st = app_raw_st[:-8]
            app_raw_st.insert(0, 'stock', stock)
      return app_raw_st

In [ ]:
def main_scrape_multi(stocks):
    return pd.concat([main_scraper(stock) for stock in stocks], sort=False)

In [ ]:
# Space to code the IPO's Scraper


In [ ]:
stocks = ['MSFT', 'T', 'AAPL', 'AMD', 'GOOGL', 'AMZN', 'CSCO', 'KO', 'HD', 'HPE', 'INTC', 'MCD', 'NFLX', 'NVDA', 'QCOM', 'TMUS', 'WMT', 'TWTR', 'EBAY', 'FB', 'FDX', 'NOK', 'CVX', 'GM', 'HAS', 'HST', 'K', 'MET', 'MU', 'NKE', 'NLOK', 'ORCL', 'PEP', 'RL', 'SBUX', 'DIS', 'F', 'PBR', 'XOM', 'AAL', 'BA', 'SONY', 'SHOP', 'SAP', 'ADBE', 'TSM', 'BABA', 'AMX', 'VZ', 'SPG', 'INVH', 'CCI', 'LPL', 'FLEX', 'DBX', 'GDDY', 'PAGS', 'CRM', 'BIDU', 'CMCM', 'CX', 'TV', 'HERDEZ.MX', 'NTDOY']
main_df_combined = main_scrape_multi(stocks)
# Removed stocks: TM, ABNB, MCHP

Attempting to scrape data for MSFT

(function (root) {
/* -- Data -- */
root.App || (root.App = {});
root.App.now = 1622041309666;
root.App.main = {"context":{"dispatcher":{"stores":{"PageStore":{"currentPageName":"quote","currentEvent":{"eventName":"NEW_PAGE_SUCCESS"},"currentRenderTargetId":"default","pagesConfigRaw":{"base":{"quote":{"layout":{"bundleName":"yahoodotcom-layout.TwoColumnLayout","name":"TwoColumnLayout","config":{"enableHeaderCollapse":true,"additionalBodyWrapperClasses":"Bgc($layoutBgColor)!","contentWrapperClas
put":{"strings":1},"tdv2-applet-sponsored-moments":{"strings":1},"tdv2-applet-stream":{"strings":1},"tdv2-applet-stream-hero":{"strings":1},"tdv2-applet-swisschamp":{"strings":1},"tdv2-applet-uh":{"strings":1},"tdv2-applet-userintent":{"strings":1},"tdv2-applet-video-lightbox":{"strings":1},"tdv2-applet-video-modal":{"strings":1},"tdv2-wafer-adfeedback":{"strings":1},"tdv2-wafer-header":{"strings":1},"yahoodotcom-layout":{"strings":1}}},"options":{"defaultBund

In [ ]:
main_df_combined = main_df_combined[['stock', 'endDate', 'netIncome', 'totalRevenue', 'ebit', 'researchDevelopment', 'operatingIncome', 'interestExpense', 'totalAssets', 'totalLiab', 'totalCurrentLiabilities', 'totalCurrentAssets', 'repurchaseOfStock']]
main_df_combined

,stock,endDate,netIncome,totalRevenue,ebit,researchDevelopment,operatingIncome,interestExpense,totalAssets,totalLiab,totalCurrentLiabilities,totalCurrentAssets,repurchaseOfStock
0,MSFT,1593475200,4.428100e+10,1.430150e+11,5.295900e+10,1.926900e+10,5.295900e+10,-2.591000e+09,3.013110e+11,1.830070e+11,7.231000e+10,1.819150e+11,-2.296800e+10
1,MSFT,1561852800,3.924000e+10,1.258430e+11,4.295900e+10,1.687600e+10,4.295900e+10,-2.686000e+09,2.865560e+11,1.842260e+11,6.942000e+10,1.755520e+11,-1.954300e+10
2,MSFT,1530316800,1.657100e+10,1.103600e+11,3.505800e+10,1.472600e+10,3.505800e+10,-2.733000e+09,2.588480e+11,1.761300e+11,5.848800e+10,1.696620e+11,-1.072100e+10
3,MSFT,1498780800,2.548900e+10,9.657100e+10,2.933100e+10,1.303700e+10,2.933100e+10,-2.222000e+09,2.503120e+11,1.626010e+11,5.574500e+10,1.626960e+11,-1.178800e+10
0,T,1609372800,-5.176000e+09,1.717600e+11,2.565600e+10,NaN,2.565600e+10,-7.925000e+09,5.257610e+11,3.465210e+11,6.343800e+10,5.200800e+10,-5.498000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,HERDEZ.MX,1514678400,1.046111e+09,2.006492e+10,2.812898e+09,NaN,2.812898e+09,-5.734910e+08,2.784602e+10,1.050422e+10,3.123141e+09,8.254989e+09,-1.551190e+08
0,NTDOY,1585612800,2.586410e+11,1.308519e+12,3.523710e+11,8.412700e+10,3.523710e+11,NaN,1.934087e+12,3.931870e+11,3.556830e+11,1.501583e+12,-4.300000e+07
1,NTDOY,1553990400,1.940090e+11,1.200560e+12,2.497020e+11,6.962400e+10,2.497020e+11,NaN,1.690304e+12,2.755060e+11,2.450090e+11,1.344972e+12,-3.103800e+10
2,NTDOY,1522454400,1.395900e+11,1.055682e+12,1.775580e+11,6.399900e+10,1.775580e+11,NaN,1.633748e+12,3.101730e+11,2.780760e+11,1.276764e+12,-7.800000e+07


In [ ]:
main_df_combined.isna().sum().head(60)

stock                        0
endDate                      0
netIncome                    0
totalRevenue                 0
ebit                         0
researchDevelopment        120
operatingIncome              0
interestExpense              9
totalAssets                  0
totalLiab                    0
totalCurrentLiabilities      0
totalCurrentAssets           0
repurchaseOfStock           44
dtype: int64

Stock Price Scraper

In [ ]:
initial_prices = []
for row in range(len(main_df_combined)):

  try:
    per0 = main_df_combined.iloc[row, 1] 
    startperiod = per0 - 1123200 # 13 days before ending quarter (not necesarilly presentation date)
    FinDate = startperiod + 864000 # 10 days range to scrape, selecting just the first price
    stockrow_init = main_df_combined.iloc[row, 0] 
    
    stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'
    params = {
        'period1':startperiod,
        'period2':FinDate,
        'interval':'1d',
        'events':'history'
    }

    
    response = requests.get(stock_url.format(stockrow_init), params=params)

    response.text

    file = StringIO(response.text)
    reader = csv.reader(file)
    data = list(reader)

    initial_price = data[1][4]
    initial_prices.append(initial_price)
  except IndexError:
    initial_price = 0
    initial_prices.append(initial_price)

In [ ]:
initial_prices

['194.240005',
 '132.850006',
 '100.860001',
 '70.870003',
 '29.400000',
 '38.740002',
 '29.750000',
 '38.459999',
 '115.360001',
 '54.974998',
 '54.470001',
 '39.667500',
 '94.779999',
 '42.349998',
 '18.830000',
 '10.980000',
 '1726.219971',
 '1351.910034',
 '1043.410034',
 '1085.089966',
 '3201.649902',
 '1784.030029',
 '1551.479980',
 '1190.579956',
 '45.930000',
 '58.049999',
 '42.500000',
 '31.500000',
 '53.740002',
 '53.910000',
 '48.320000',
 '45.930000',
 '273.190002',
 '232.949997',
 '177.110001',
 '196.309998',
 '9.350000',
 '15.350000',
 '15.330000',
 '14.700000',
 '50.470001',
 '57.700001',
 '47.080002',
 '46.259998',
 '215.080002',
 '195.630005',
 '179.710007',
 '174.199997',
 '534.450012',
 '320.799988',
 '270.940002',
 '190.419998',
 '521.010010',
 '251.979996',
 '150.440002',
 '220.110001',
 '113.459999',
 '78.040001',
 '73.239998',
 '50.570000',
 '131.919998',
 '76.309998',
 '64.779999',
 '63.470001',
 '143.389999',
 '115.589996',
 '97.730003',
 '104.300003',
 '55.869

In [ ]:
        period0 = main_df_combined.iloc[5, 1] 
        startDate = period0 + 6652800
        EndDate = startDate + 864000
        stockrow_fn = main_df_combined.iloc[1, 0] 

        stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'
        params = {
            'period1':1620604800,
            'period2':1621987200,
            'interval':'1d',
            'events':'history'
        }

        response = requests.get(stock_url.format(stockrow_fn), params=params)
        response.text

        file = StringIO(response.text)
        reader = csv.reader(file)
        data = list(reader)

        data

        #final_price = data[1][4]
        #final_price

In [ ]:
final_prices = []

for row in range(len(main_df_combined)):

      try: 
        period0 = main_df_combined.iloc[row, 1] 
        startDate = period0 + 6652800 # 77 days after quarter ending (not necessary statms presentation date)
        EndDate = startDate + 864000  # 10 days range to scrape, selecting just the first price
        stockrow_fn = main_df_combined.iloc[row, 0] 

        stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'
        params = {
            'period1':startDate,
            'period2':EndDate,
            'interval':'1d',
            'events':'history'
        }

        response = requests.get(stock_url.format(stockrow_fn), params=params)
        response.text

        file = StringIO(response.text)
        reader = csv.reader(file)
        data = list(reader)

        final_price = data[1][4]
        final_prices.append(final_price)
      except IndexError:
        final_price = 0
        final_prices.append(final_price)
  # Reasons for this code to outout error: 
  # 1.- Stock's last statements presentation 2.5 months or less ago
  # 2.- stocks younger than 4 years being public
  #

In [ ]:
final_prices

['208.779999',
 '136.330002',
 '112.139999',
 '75.309998',
 '30.010000',
 '33.740002',
 '30.799999',
 '36.580002',
 '121.779999',
 '69.964996',
 '40.985001',
 '44.105000',
 '82.500000',
 '38.709999',
 '23.250000',
 '11.430000',
 '2021.339966',
 '1118.060059',
 '1188.550049',
 '1100.069946',
 '3027.989990',
 '1807.839966',
 '1742.150024',
 '1544.930054',
 '40.360001',
 '46.049999',
 '44.669998',
 '33.540001',
 '50.570000',
 '47.180000',
 '45.410000',
 '43.259998',
 '326.850006',
 '206.050003',
 '204.779999',
 '174.429993',
 '12.310000',
 '15.770000',
 '14.320000',
 '15.810000',
 '63.790001',
 '44.610001',
 '54.099998',
 '50.830002',
 '222.899994',
 '147.619995',
 '183.949997',
 '159.009995',
 '504.790009',
 '319.750000',
 '363.440002',
 '313.480011',
 '614.469971',
 '269.850006',
 '184.699997',
 '231.490005',
 '146.289993',
 '88.430000',
 '57.000000',
 '65.180000',
 '124.910004',
 '78.349998',
 '73.680000',
 '63.220001',
 '139.710007',
 '132.119995',
 '103.180000',
 '87.570000',
 '66.72

In [ ]:
main_df_combined['Initial Prices'] = np.asarray(initial_prices).flatten()
main_df_combined['Final Prices'] = np.asarray(final_prices).flatten()
main_df_combined

,stock,endDate,netIncome,totalRevenue,ebit,researchDevelopment,operatingIncome,interestExpense,totalAssets,totalLiab,totalCurrentLiabilities,totalCurrentAssets,repurchaseOfStock,Initial Prices,Final Prices
0,MSFT,1593475200,4.428100e+10,1.430150e+11,5.295900e+10,1.926900e+10,5.295900e+10,-2.591000e+09,3.013110e+11,1.830070e+11,7.231000e+10,1.819150e+11,-2.296800e+10,194.240005,208.779999
1,MSFT,1561852800,3.924000e+10,1.258430e+11,4.295900e+10,1.687600e+10,4.295900e+10,-2.686000e+09,2.865560e+11,1.842260e+11,6.942000e+10,1.755520e+11,-1.954300e+10,132.850006,136.330002
2,MSFT,1530316800,1.657100e+10,1.103600e+11,3.505800e+10,1.472600e+10,3.505800e+10,-2.733000e+09,2.588480e+11,1.761300e+11,5.848800e+10,1.696620e+11,-1.072100e+10,100.860001,112.139999
3,MSFT,1498780800,2.548900e+10,9.657100e+10,2.933100e+10,1.303700e+10,2.933100e+10,-2.222000e+09,2.503120e+11,1.626010e+11,5.574500e+10,1.626960e+11,-1.178800e+10,70.870003,75.309998
0,T,1609372800,-5.176000e+09,1.717600e+11,2.565600e+10,NaN,2.565600e+10,-7.925000e+09,5.257610e+11,3.465210e+11,6.343800e+10,5.200800e+10,-5.498000e+09,29.400000,30.010000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,HERDEZ.MX,1514678400,1.046111e+09,2.006492e+10,2.812898e+09,NaN,2.812898e+09,-5.734910e+08,2.784602e+10,1.050422e+10,3.123141e+09,8.254989e+09,-1.551190e+08,45.730000,46.980000
0,NTDOY,1585612800,2.586410e+11,1.308519e+12,3.523710e+11,8.412700e+10,3.523710e+11,NaN,1.934087e+12,3.931870e+11,3.556830e+11,1.501583e+12,-4.300000e+07,40.709999,55.700001
1,NTDOY,1553990400,1.940090e+11,1.200560e+12,2.497020e+11,6.962400e+10,2.497020e+11,NaN,1.690304e+12,2.755060e+11,2.450090e+11,1.344972e+12,-3.103800e+10,35.889999,43.180000
2,NTDOY,1522454400,1.395900e+11,1.055682e+12,1.775580e+11,6.399900e+10,1.775580e+11,NaN,1.633748e+12,3.101730e+11,2.780760e+11,1.276764e+12,-7.800000e+07,56.400002,41.650002


Converting Date Format

In [ ]:
df = main_df_combined
df_enddate = df['endDate']
df_enddate

0    1593475200
1    1561852800
2    1530316800
3    1498780800
0    1609372800
        ...    
3    1514678400
0    1585612800
1    1553990400
2    1522454400
3    1490918400
Name: endDate, Length: 256, dtype: int64

In [ ]:
len(df_enddate)

256

In [ ]:
df

,stock,endDate,netIncome,totalRevenue,ebit,researchDevelopment,operatingIncome,interestExpense,totalAssets,totalLiab,totalCurrentLiabilities,totalCurrentAssets,repurchaseOfStock,Initial Prices,Final Prices
0,MSFT,1593475200,4.428100e+10,1.430150e+11,5.295900e+10,1.926900e+10,5.295900e+10,-2.591000e+09,3.013110e+11,1.830070e+11,7.231000e+10,1.819150e+11,-2.296800e+10,194.240005,208.779999
1,MSFT,1561852800,3.924000e+10,1.258430e+11,4.295900e+10,1.687600e+10,4.295900e+10,-2.686000e+09,2.865560e+11,1.842260e+11,6.942000e+10,1.755520e+11,-1.954300e+10,132.850006,136.330002
2,MSFT,1530316800,1.657100e+10,1.103600e+11,3.505800e+10,1.472600e+10,3.505800e+10,-2.733000e+09,2.588480e+11,1.761300e+11,5.848800e+10,1.696620e+11,-1.072100e+10,100.860001,112.139999
3,MSFT,1498780800,2.548900e+10,9.657100e+10,2.933100e+10,1.303700e+10,2.933100e+10,-2.222000e+09,2.503120e+11,1.626010e+11,5.574500e+10,1.626960e+11,-1.178800e+10,70.870003,75.309998
0,T,1609372800,-5.176000e+09,1.717600e+11,2.565600e+10,NaN,2.565600e+10,-7.925000e+09,5.257610e+11,3.465210e+11,6.343800e+10,5.200800e+10,-5.498000e+09,29.400000,30.010000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,HERDEZ.MX,1514678400,1.046111e+09,2.006492e+10,2.812898e+09,NaN,2.812898e+09,-5.734910e+08,2.784602e+10,1.050422e+10,3.123141e+09,8.254989e+09,-1.551190e+08,45.730000,46.980000
0,NTDOY,1585612800,2.586410e+11,1.308519e+12,3.523710e+11,8.412700e+10,3.523710e+11,NaN,1.934087e+12,3.931870e+11,3.556830e+11,1.501583e+12,-4.300000e+07,40.709999,55.700001
1,NTDOY,1553990400,1.940090e+11,1.200560e+12,2.497020e+11,6.962400e+10,2.497020e+11,NaN,1.690304e+12,2.755060e+11,2.450090e+11,1.344972e+12,-3.103800e+10,35.889999,43.180000
2,NTDOY,1522454400,1.395900e+11,1.055682e+12,1.775580e+11,6.399900e+10,1.775580e+11,NaN,1.633748e+12,3.101730e+11,2.780760e+11,1.276764e+12,-7.800000e+07,56.400002,41.650002


In [ ]:
import datetime
new_date_col = []
for D in df_enddate:

    timestamp = datetime.datetime.fromtimestamp(D)
    date_format = timestamp.strftime('%Y-%m-%d')
    new_date_col.append(date_format)

In [ ]:
new_date_col

In [ ]:
df['endDate'] = new_date_col
df

,stock,endDate,netIncome,totalRevenue,ebit,researchDevelopment,operatingIncome,interestExpense,totalAssets,totalLiab,totalCurrentLiabilities,totalCurrentAssets,repurchaseOfStock,Initial Prices,Final Prices
0,MSFT,2020-06-30,4.428100e+10,1.430150e+11,5.295900e+10,1.926900e+10,5.295900e+10,-2.591000e+09,3.013110e+11,1.830070e+11,7.231000e+10,1.819150e+11,-2.296800e+10,194.240005,208.779999
1,MSFT,2019-06-30,3.924000e+10,1.258430e+11,4.295900e+10,1.687600e+10,4.295900e+10,-2.686000e+09,2.865560e+11,1.842260e+11,6.942000e+10,1.755520e+11,-1.954300e+10,132.850006,136.330002
2,MSFT,2018-06-30,1.657100e+10,1.103600e+11,3.505800e+10,1.472600e+10,3.505800e+10,-2.733000e+09,2.588480e+11,1.761300e+11,5.848800e+10,1.696620e+11,-1.072100e+10,100.860001,112.139999
3,MSFT,2017-06-30,2.548900e+10,9.657100e+10,2.933100e+10,1.303700e+10,2.933100e+10,-2.222000e+09,2.503120e+11,1.626010e+11,5.574500e+10,1.626960e+11,-1.178800e+10,70.870003,75.309998
0,T,2020-12-31,-5.176000e+09,1.717600e+11,2.565600e+10,NaN,2.565600e+10,-7.925000e+09,5.257610e+11,3.465210e+11,6.343800e+10,5.200800e+10,-5.498000e+09,29.400000,30.010000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,HERDEZ.MX,2017-12-31,1.046111e+09,2.006492e+10,2.812898e+09,NaN,2.812898e+09,-5.734910e+08,2.784602e+10,1.050422e+10,3.123141e+09,8.254989e+09,-1.551190e+08,45.730000,46.980000
0,NTDOY,2020-03-31,2.586410e+11,1.308519e+12,3.523710e+11,8.412700e+10,3.523710e+11,NaN,1.934087e+12,3.931870e+11,3.556830e+11,1.501583e+12,-4.300000e+07,40.709999,55.700001
1,NTDOY,2019-03-31,1.940090e+11,1.200560e+12,2.497020e+11,6.962400e+10,2.497020e+11,NaN,1.690304e+12,2.755060e+11,2.450090e+11,1.344972e+12,-3.103800e+10,35.889999,43.180000
2,NTDOY,2018-03-31,1.395900e+11,1.055682e+12,1.775580e+11,6.399900e+10,1.775580e+11,NaN,1.633748e+12,3.101730e+11,2.780760e+11,1.276764e+12,-7.800000e+07,56.400002,41.650002


In [ ]:
df.isna().sum()
#main_df_combined.isna().sum().head(60)

stock                        0
endDate                      0
netIncome                    0
totalRevenue                 0
ebit                         0
researchDevelopment        120
operatingIncome              0
interestExpense              9
totalAssets                  0
totalLiab                    0
totalCurrentLiabilities      0
totalCurrentAssets           0
repurchaseOfStock           44
Initial Prices               0
Final Prices                 0
dtype: int64

In [ ]:
df.to_csv('firstaccountsnprices.csv', index= False)

LXML Scraper for Missing Accounts

In [ ]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

In [ ]:
def get_page(url):
    # Set up the request headers that we're going to use, to simulate
    # a request by the Chrome browser. Simulating a request from a browser
    # is generally good practice when building a scraper
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
    }
 
    return requests.get(url, headers=headers)
 
def parse_rows(table_rows):
    parsed_rows = []
 
    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")
 
        none_count = 0
 
        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1
 
        if (none_count < 4):
            parsed_rows.append(parsed_row)
            
    return pd.DataFrame(parsed_rows)
 
def clean_data(df):
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names
    
    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    
    numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)
 
    for column_index in range(1, len(df.columns)): # Take all columns, except the first (which is the 'Date' column)
        df.iloc[:,column_index] = df.iloc[:,column_index].str.replace(',', '') # Remove the thousands separator
        df.iloc[:,column_index] = df.iloc[:,column_index].astype(np.float64) # Convert the column to float64
        
    return df
 
 
 
def scrape_table(url):
    # Fetch the page that we're going to parse
    page = get_page(url);
 
    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)
 
    # Fetch all div elements which have class 'D(tbr)'
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
    
    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    assert len(table_rows) > 0
    
    df = parse_rows(table_rows)
    df = clean_data(df)
        
    return df

In [ ]:
def scrape(symbol):
    print('Attempting to scrape data for ' + symbol)
 
    df_balance_sheet = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol)
    df_balance_sheet = df_balance_sheet.set_index('Date')
 
    df_income_statement = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol)
    df_income_statement = df_income_statement.set_index('Date')
    
    df_cash_flow = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p=' + symbol)
    df_cash_flow = df_cash_flow.set_index('Date')
    
    df_joined = df_balance_sheet \
        .join(df_income_statement, on='Date', how='outer', rsuffix=' - Income Statement') \
        .join(df_cash_flow, on='Date', how='outer', rsuffix=' - Cash Flow') \
        .dropna(axis=1, how='all') \
        .reset_index()
            
    df_joined.insert(1, 'Symbol', symbol)
    
    return df_joined

In [ ]:
def scrape_multi(symbols):
    return pd.concat([scrape(symbol) for symbol in symbols], sort=False)

In [ ]:
symbols = ['EBAY', 'FB', 'FDX', 'NOK', 'CVX', 'GM', 'HAS', 'HST', 'K', 'MET', 'MU', 'NKE', 'NLOK', 'ORCL', 'PEP', 'RL', 'SBUX', 'DIS', 'F', 'PBR', 'XOM', 'AAL', 'BA', 'SONY', 'SHOP', 'SAP', 'ADBE', 'TSM', 'BABA', 'AMX', 'VZ', 'SPG', 'INVH', 'CCI', 'LPL', 'FLEX', 'DBX', 'GDDY', 'PAGS', 'CRM', 'BIDU', 'CMCM', 'CX', 'TV', 'HERDEZ.MX']
 #'MSFT', 'T', 'AAPL', 'AMD', 'GOOGL', 'AMZN', 'CSCO', 'KO', 'HD', 'HPE', 'INTC', 'MCD', 'NFLX', 'NVDA', 'QCOM', 'TMUS', 'WMT', 'TWTR',            
#'NTDOY'
df_combined = scrape_multi(symbols)

In [ ]:
df_combined = df_combined[['Symbol', 'Date', 'Ordinary Shares Number', 'Total Debt', 'Free Cash Flow']]
df_combined

,Symbol,Date,Ordinary Shares Number,Total Debt,Free Cash Flow
0,EBAY,12/31/2020,684000.0,8133000.0,1925000.0
1,EBAY,12/31/2019,795888.0,8252000.0,2560000.0
2,EBAY,12/31/2018,915000.0,9231000.0,2007000.0
3,EBAY,12/31/2017,1029000.0,10015000.0,2480000.0
4,EBAY,ttm,NaN,NaN,2389000.0
...,...,...,...,...,...
0,HERDEZ.MX,12/31/2020,341712.0,10293637.0,2121697.0
1,HERDEZ.MX,12/31/2019,380578.0,8859043.0,1670310.0
2,HERDEZ.MX,12/31/2018,421611.0,6638259.0,1591127.0
3,HERDEZ.MX,12/31/2017,428549.0,6350785.0,1327645.0


In [ ]:
complementdf = pd.concat([first_complementDF, second_complemtetDF])
complementdf

In [ ]:
complementdf.to_csv('secondtaccountsnpricesJOINED.csv', index= False)

Scrapers Data mismatches Detection

In [ ]:
from google.colab import files
import io
uploaded = files.upload()

Dismatches1_df = pd.read_csv(io.BytesIO(uploaded['firstaccountsnprices.csv']))

Dismatches1_df[Dismatches1_df['endDate'].str.contains("2021")]
#FLEX
#BABA

In [ ]:
from google.colab import files
import io
uploaded = files.upload()

Dismatches2_df = pd.read_csv(io.BytesIO(uploaded['secondtaccountsnpricesJOINED.csv']))

Dismatches2_df[Dismatches2_df['Date'].str.contains("2021")]
#K

In [ ]:
#comparison_column = np.where(Dismatches1_df["stock"] == Dismatches2_df["Symbol"], True, False)
compar_1 = Dismatches1_df[Dismatches1_df['endDate'].str.contains("2021")]
compar_1 = compar_1[['stock','endDate']]
compar_2 = Dismatches2_df[Dismatches2_df['Date'].str.contains("2021")]
compar_2 = compar_2[['Symbol','Date']]
comparison = compar_1.merge(compar_2, how= 'outer', left_on= 'stock', right_on= 'Symbol', indicator = True)
comparison = comparison[comparison['_merge'].str.contains("_only")]
comparison

,stock,endDate,Symbol,Date,_merge
3,K,2021-01-02,NaN,NaN,left_only
8,NaN,NaN,BABA,3/31/2021,right_only
9,NaN,NaN,FLEX,3/31/2021,right_only


In [ ]:
dism_stocks_list = ["K", "FLEX", "BABA"]
dis_complet = Dismatches1_df[Dismatches1_df.iloc[:,0].isin(dism_stocks_list)]
#df1 = df1[~df1.iloc[:, 0].isin(words)]
dis_complet = dis_complet[['stock','endDate']]


In [ ]:
dism_stocks_list = ["K", "FLEX", "BABA"]
dis_complet2 = Dismatches2_df[Dismatches2_df.iloc[:,0].isin(dism_stocks_list)]
#df1 = df1[~df1.iloc[:, 0].isin(words)]
dis_complet2 = dis_complet2[['Symbol','Date']]

In [ ]:
compare_mismatches = pd.concat([dis_complet, dis_complet2],axis=1)
compare_mismatches

,stock,endDate,Symbol,Date
104,K,2021-01-02,K,12/31/2020
105,K,2019-12-28,K,12/31/2019
106,K,2018-12-29,K,12/31/2018
107,K,2017-12-30,K,12/31/2017
184,BABA,2020-03-31,BABA,3/31/2021
185,BABA,2019-03-31,BABA,3/31/2020
186,BABA,2018-03-31,BABA,3/31/2019
187,BABA,2017-03-31,BABA,3/31/2018
212,FLEX,2020-03-31,FLEX,3/31/2021
213,FLEX,2019-03-31,FLEX,3/31/2020


In [ ]:
#data = data.drop(labels=[1,15,20], axis=0)
Dismatches1_removed = Dismatches1_df.drop(labels=[187,215])
Dismatches2_removed = Dismatches2_df.drop(labels=[184,212])

In [ ]:
#WHAT IS NEXT IS TO CONCAT BOTH DATAFRAMES AND VERIFY IF IS EQUAL TO THE EXCEL MADE ONE

In [ ]:
fina_test = pd.concat([cleaned_Df, df_combined], axis=1)   
fina_test